In [1]:
from transformers import LlamaTokenizer, LlamaForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import OnlinePDFLoader
from langchain.vectorstores import Chroma

import os
import torch

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-04-08 07:02:27.874360: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# used_model = "decapoda-research/llama-7b-hf"
used_model = "chavinlo/gpt4-x-alpaca"
device_map = {"": int(os.environ.get("LOCAL_RANK") or 0)}
create_db=True
persist_directory = "/app/VectorStore"

In [3]:
tokenizer = LlamaTokenizer.from_pretrained(used_model)
base_model = LlamaForCausalLM.from_pretrained(
    used_model,
    load_in_8bit=True,
    device_map=device_map,
    offload_folder="/app/models_gpt/",
    #low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
)
pipe = pipeline(
    "text-generation",
    model=base_model, 
    tokenizer=tokenizer, 
    max_length=4000,
    temperature=0.6,
    top_p=0.95,
    repetition_penalty=1.2
)
llm = HuggingFacePipeline(pipeline=pipe)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.
Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/bitsandbytes/functional.py:158: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  else: return ct.c_void_p(A.data.storage().data_ptr())
Loading checkpoint shards: 100%|██████████| 33/33 [01:13<00:00,  2.24s/it]


In [4]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

embeddings = HuggingFaceInstructEmbeddings(
    query_instruction="Represent the query for retrieval: ",
    model_name = "hkunlp/instructor-large",
)

## Only use HF Hub for exploration
#from langchain.embeddings import HuggingFaceEmbeddings
#embeddings = HuggingFaceEmbeddings()

load INSTRUCTOR_Transformer
max_seq_length  512


In [5]:
loader = OnlinePDFLoader("https://www-axa-com.cdn.axa-contento-118412.eu/www-axa-com/85ec0278-bf2f-4392-94b9-c086717fa8f6_axa_urd2022_accessible_va.pdf")

In [6]:
if create_db:
    text_splitter = CharacterTextSplitter(chunk_size=450, chunk_overlap=25)
    vectorstore_type = Chroma

    index = VectorstoreIndexCreator(
        embedding=embeddings, 
        text_splitter= text_splitter,
        vectorstore_cls=vectorstore_type,
        vectorstore_kwargs={"persist_directory":persist_directory}
        ).from_loaders([loader])
    vectorstore=index.vectorstore
    vectorstore.persist()

else:
    # Load Store
    vectorstore = Chroma(persist_directory=persist_directory, embedding_function=embeddings)

detectron2 is not installed. Cannot use the hi_res partitioning strategy. Falling back to partitioning with the fast strategy.
Using embedded DuckDB with persistence: data will be stored in: /app/VectorStore


: 

: 

In [ ]:
docs = vectorstore.as_retriever().get_relevant_documents("what are the enviroment goals of AXA?")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_5357/1650839913.py:1 in <module>                                                  │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_5357/1650839913.py'                         │
│                                                                                                  │
│ /usr/local/lib/python3.8/dist-packages/langchain/vectorstores/base.py:154 in                     │
│ get_relevant_documents                                                                           │
│                                                                                                  │
│   151 │                                                                                          │
│   152 │   def get_relevant_documents(self, query: str) -> List[Document]:                        │
│   153 │   │   if self.search_type == "similarity":                                               │
│ ❱ 154 │   │   │   docs = self.vectorstore.similarity_search(query, **self.search_kwargs)         │
│   155 │   │   elif self.search_type == "mmr":                                                    │
│   156 │   │   │   docs = self.vectorstore.max_marginal_relevance_search(                         │
│   157 │   │   │   │   query, **self.search_kwargs                                                │
│                                                                                                  │
│ /usr/local/lib/python3.8/dist-packages/langchain/vectorstores/chroma.py:136 in similarity_search │
│                                                                                                  │
│   133 │   │   Returns:                                                                           │
│   134 │   │   │   List[Document]: List of documents most similar to the query text.              │
│   135 │   │   """                                                                                │
│ ❱ 136 │   │   docs_and_scores = self.similarity_search_with_score(query, k, filter=filter)       │
│   137 │   │   return [doc for doc, _ in docs_and_scores]                                         │
│   138 │                                                                                          │
│   139 │   def similarity_search_by_vector(                                                       │
│                                                                                                  │
│ /usr/local/lib/python3.8/dist-packages/langchain/vectorstores/chroma.py:182 in                   │
│ similarity_search_with_score                                                                     │
│                                                                                                  │
│   179 │   │   │   )                                                                              │
│   180 │   │   else:                                                                              │
│   181 │   │   │   query_embedding = self._embedding_function.embed_query(query)                  │
│ ❱ 182 │   │   │   results = self._collection.query(                                              │
│   183 │   │   │   │   query_embeddings=[query_embedding], n_results=k, where=filter              │
│   184 │   │   │   )                                                                              │
│   185                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.8/dist-packages/chromadb/api/models/Collection.py:202 in query            │
│                                                                                                  │
│   199 │   │   if where_document is None:                   

In [ ]:
docs

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_5357/775672773.py:1 in <module>                                                   │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_5357/775672773.py'                          │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'docs' is not defined

In [ ]:
from langchain.chains import RetrievalQA
chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectorstore.as_retriever(), input_key="question")

In [ ]:
query = "What are the AXA Goals for the future?"
chain.run(query)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_5357/2593917990.py:2 in <module>                                                  │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_5357/2593917990.py'                         │
│                                                                                                  │
│ /usr/local/lib/python3.8/dist-packages/langchain/chains/base.py:213 in run                       │
│                                                                                                  │
│   210 │   │   if args and not kwargs:                                                            │
│   211 │   │   │   if len(args) != 1:                                                             │
│   212 │   │   │   │   raise ValueError("`run` supports only one positional argument.")           │
│ ❱ 213 │   │   │   return self(args[0])[self.output_keys[0]]                                      │
│   214 │   │                                                                                      │
│   215 │   │   if kwargs and not args:                                                            │
│   216 │   │   │   return self(kwargs)[self.output_keys[0]]                                       │
│                                                                                                  │
│ /usr/local/lib/python3.8/dist-packages/langchain/chains/base.py:116 in __call__                  │
│                                                                                                  │
│   113 │   │   │   outputs = self._call(inputs)                                                   │
│   114 │   │   except (KeyboardInterrupt, Exception) as e:                                        │
│   115 │   │   │   self.callback_manager.on_chain_error(e, verbose=self.verbose)                  │
│ ❱ 116 │   │   │   raise e                                                                        │
│   117 │   │   self.callback_manager.on_chain_end(outputs, verbose=self.verbose)                  │
│   118 │   │   return self.prep_outputs(inputs, outputs, return_only_outputs)                     │
│   119                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.8/dist-packages/langchain/chains/base.py:113 in __call__                  │
│                                                                                                  │
│   110 │   │   │   verbose=self.verbose,                                                          │
│   111 │   │   )                                                                                  │
│   112 │   │   try:                                                                               │
│ ❱ 113 │   │   │   outputs = self._call(inputs)                                                   │
│   114 │   │   except (KeyboardInterrupt, Exception) as e:                                        │
│   115 │   │   │   self.callback_manager.on_chain_error(e, verbose=self.verbose)                  │
│   116 │   │   │   raise e                                                                        │
│                                                                                                  │
│ /usr/local/lib/python3.8/dist-packages/langchain/chains/retrieval_qa/base.py:109 in _call        │
│                                                                                                  │
│   106 │   │   """                                                                                │
│   107 │   │   question = inputs[self.input_key]                                                  │
│   108 │   │                                                

In [ ]:
docs = vectorstore.as_retriever().get_relevant_documents(query)
print(docs)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_5357/441866326.py:1 in <module>                                                   │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_5357/441866326.py'                          │
│                                                                                                  │
│ /usr/local/lib/python3.8/dist-packages/langchain/vectorstores/base.py:154 in                     │
│ get_relevant_documents                                                                           │
│                                                                                                  │
│   151 │                                                                                          │
│   152 │   def get_relevant_documents(self, query: str) -> List[Document]:                        │
│   153 │   │   if self.search_type == "similarity":                                               │
│ ❱ 154 │   │   │   docs = self.vectorstore.similarity_search(query, **self.search_kwargs)         │
│   155 │   │   elif self.search_type == "mmr":                                                    │
│   156 │   │   │   docs = self.vectorstore.max_marginal_relevance_search(                         │
│   157 │   │   │   │   query, **self.search_kwargs                                                │
│                                                                                                  │
│ /usr/local/lib/python3.8/dist-packages/langchain/vectorstores/chroma.py:136 in similarity_search │
│                                                                                                  │
│   133 │   │   Returns:                                                                           │
│   134 │   │   │   List[Document]: List of documents most similar to the query text.              │
│   135 │   │   """                                                                                │
│ ❱ 136 │   │   docs_and_scores = self.similarity_search_with_score(query, k, filter=filter)       │
│   137 │   │   return [doc for doc, _ in docs_and_scores]                                         │
│   138 │                                                                                          │
│   139 │   def similarity_search_by_vector(                                                       │
│                                                                                                  │
│ /usr/local/lib/python3.8/dist-packages/langchain/vectorstores/chroma.py:182 in                   │
│ similarity_search_with_score                                                                     │
│                                                                                                  │
│   179 │   │   │   )                                                                              │
│   180 │   │   else:                                                                              │
│   181 │   │   │   query_embedding = self._embedding_function.embed_query(query)                  │
│ ❱ 182 │   │   │   results = self._collection.query(                                              │
│   183 │   │   │   │   query_embeddings=[query_embedding], n_results=k, where=filter              │
│   184 │   │   │   )                                                                              │
│   185                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.8/dist-packages/chromadb/api/models/Collection.py:202 in query            │
│                                                                                                  │
│   199 │   │   if where_document is None:                   

In [ ]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
chain_w_sources = load_qa_with_sources_chain(llm, chain_type="stuff")
chain_w_sources({"input_documents": vectorstore.as_retriever().get_relevant_documents(query), "question": query}, return_only_outputs=True)


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_5357/1388586871.py:3 in <module>                                                  │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_5357/1388586871.py'                         │
│                                                                                                  │
│ /usr/local/lib/python3.8/dist-packages/langchain/vectorstores/base.py:154 in                     │
│ get_relevant_documents                                                                           │
│                                                                                                  │
│   151 │                                                                                          │
│   152 │   def get_relevant_documents(self, query: str) -> List[Document]:                        │
│   153 │   │   if self.search_type == "similarity":                                               │
│ ❱ 154 │   │   │   docs = self.vectorstore.similarity_search(query, **self.search_kwargs)         │
│   155 │   │   elif self.search_type == "mmr":                                                    │
│   156 │   │   │   docs = self.vectorstore.max_marginal_relevance_search(                         │
│   157 │   │   │   │   query, **self.search_kwargs                                                │
│                                                                                                  │
│ /usr/local/lib/python3.8/dist-packages/langchain/vectorstores/chroma.py:136 in similarity_search │
│                                                                                                  │
│   133 │   │   Returns:                                                                           │
│   134 │   │   │   List[Document]: List of documents most similar to the query text.              │
│   135 │   │   """                                                                                │
│ ❱ 136 │   │   docs_and_scores = self.similarity_search_with_score(query, k, filter=filter)       │
│   137 │   │   return [doc for doc, _ in docs_and_scores]                                         │
│   138 │                                                                                          │
│   139 │   def similarity_search_by_vector(                                                       │
│                                                                                                  │
│ /usr/local/lib/python3.8/dist-packages/langchain/vectorstores/chroma.py:182 in                   │
│ similarity_search_with_score                                                                     │
│                                                                                                  │
│   179 │   │   │   )                                                                              │
│   180 │   │   else:                                                                              │
│   181 │   │   │   query_embedding = self._embedding_function.embed_query(query)                  │
│ ❱ 182 │   │   │   results = self._collection.query(                                              │
│   183 │   │   │   │   query_embeddings=[query_embedding], n_results=k, where=filter              │
│   184 │   │   │   )                                                                              │
│   185                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.8/dist-packages/chromadb/api/models/Collection.py:202 in query            │
│                                                                                                  │
│   199 │   │   if where_document is None:                   